In [1]:
%matplotlib inline
import os
import sys
import subprocess
import numpy as np
from matplotlib import pyplot as plt
import math
from scipy.stats import special_ortho_group

In [2]:
simulator_bin='~/gdrive_perso/Toolkit/Microscope_Simulator/TEM-simulator_1.3/src/TEM-simulator'

# Parameters

In [4]:
numpart = 320
pdb     = '4v6x'
dose    = 1000
xrange = np.arange(-400,450,50)
yrange = xrange
sample_dimensions = [1200,100,100]

# Generator

In [5]:
write_crd_file(numpart, xrange=xrange, yrange=yrange, filename='simulated_micrographs/'+pdb+'_copy1.txt')
for defoc in np.arange(1,3,0.5):
        keyword  = pdb+'_defoc'+str(defoc)
        params_dictionary = fill_parameters_dictionary(pdb_file='pdbs/'+pdb+'.pdb', 
                                                       crd_file='simulated_micrographs/'+pdb+'_copy1.txt',
                                                       mrc_file='simulated_micrographs/'+keyword+'.mrc',
                                                       log_file='simulated_micrographs/'+keyword+'.log',
                                                       particle_mrcout='simulated_micrographs/'+keyword,
                                                       sample_dimensions=sample_dimensions,
                                                       defocus=defoc,
                                                       dose=1000,
                                                       noise='no')
        write_inp_file(numpart, pdb_file='pdbs/'+pdb+'.pdb', crd_file='simulated_micrographs/'+pdb+'_copy1.txt', 
                       mrc_file='simulated_micrographs/'+keyword+'.mrc', log_file='simulated_micrographs/'+keyword+'.log', 
                       filename='simulated_micrographs/'+keyword+'.ini', dict_params=params_dictionary)
        os.system('{0} {1}'.format(simulator_bin,'simulated_micrographs/'+keyword+'.ini'))
        

# Toolkit

In [3]:
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def rotationMatrixToEulerAngles(R) :
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    x = (x*180)/np.pi
    y = (y*180)/np.pi
    z = (z*180)/np.pi
    return np.array([x, y, z])

def get_rotlist(numpart):
    """ get_rotlist
    """
    rotlist = []
    for x in range(0,numpart+1):
        x = special_ortho_group.rvs(3)
        y = rotationMatrixToEulerAngles(x)
        rotlist.append(y)
    return rotlist
        
def write_crd_file(numpart, xrange=np.arange(-100,110,10), yrange=np.arange(-100,110,10), filename='crd.txt'):
    """ write_crd_file
    The table should have 6 columns. The first three columns are x, y, and z coordinates of
    the particle center. The following three columns are Euler angles for rotation around
    the z axis, then around the x axis, and again around the z axis. 
    Coordinates are in nm units, and angles in degrees.
    """
    if os.path.exists(filename):
         print(filename+" already exists.")          
    else:
        rotlist  = get_rotlist(numpart)
        crd_file = open(filename, "w")
        crd_file.write('# File created by TEM-simulator, version 1.3.\n')
        crd_file.write('{numpart}  6\n'.format(numpart=numpart))
        crd_file.write('#            x             y             z           phi         theta           psi  \n')
        l = 0
        for y in yrange:
            for x in xrange:
                if l == int(numpart):
                    break
                crd_table = {'x': x, 'y': y, 'z': 0, 'phi': rotlist[l][0], 'theta': rotlist[l][1], 'psi': rotlist[l][2]}
                crd_file.write('{0[x]:14.4f}{0[y]:14.4f}{0[z]:14.4f}{0[phi]:14.4f}{0[theta]:14.4f}{0[psi]:14.4f}\n'.format(crd_table))
                l += 1
        crd_file.close()
#
def fill_parameters_dictionary(pdb_file, crd_file, mrc_file, 
                               log_file='simulator.log', particle_mrcout=None, optics_defocout=None,
                               sample_dimensions=[1200,50,150],
                               particle_name='toto', voxel_size=0.1,
                               beam_params=[300,1.3,1000000,0], dose=None,
                               optics_params=[25000,2.1,2.2,40,2.7,0.1,3.0,0,0], defocus=None,
                               detector_params=[2000,2000,16,80,'yes',0.4,0.7,0.2,0.1,10,40], noise=None,
                               seed = -1234):
    """ fill_parameters_dictionary
        
        Parameters: default values in parentheses
        -----------
        [Mandatory]
        - pdb_file (input) : PDB file of sample 
        - crd_file (input) : Coordinates of the sample copies
        - mrc_file (output): Micrograph file
        [Optional]
        - log_file ('simulator.log'): Log file for the run
        - sample_dimensions:
            . index 0 (1200): diameter in nm
            . index 1 (50)  : thickness at center in nm
            . index 2 (150) : thickness at edge in nm.
        - voxel size (0.1)      : The size of voxels in the particle map in nm.
        - particle_name ('toto'): Name of the particle. Not very important.
        - particle_mrcout (None): if not None, volume map of sample is written.
        - beam_params:
            . index 0 (300)    : voltage in kV
            . index 1 (1.3)    : energy spread in V
            . index 2 (1000000): dose per image in e/nm**2
            . index 3 (0)      : standard deviation of dose per image
        - dose (None): if not None, overrides beam_params[2]
        - optics_params:
            . index 0 (25000) : magnification
            . index 1 (2.1)   : spherical aberration in mm
            . index 2 (2.2)   : chromatic aberration in mm
            . index 3 (40)    : diameter in um of aperture in back focal plane
            . index 4 (2.7)   : focal length in mm of primary lens
            . index 5 (0.1)   : aperture angle in mrad of the beam furnished by 
                                the condenser lens
            . index 6 (3.0)   : nominal defocus value in um
            . index 7 (0)     : standard deviation of a systematic error added 
                                to the nominal defocus, measured in um. 
                                The same error is added to the defocus of every image.
            . index 8 (0)     : standard deviation of a nonsystematic error added 
                                to the nominal defocus and the systematic error, 
                                measured in um. 
                                A new value of the error is computed for every image
        - defocus (None): if not None, overrides optics_params[6]
        - optics_defocout (None): if not None, defocus values written to file
        - detector_params: 
            . index 0  (2000) : number of pixels on detector along x axis
            . index 1  (2000) : number of pixels on detector along y axis
            . index 2  (16)   : physical pixel size in um
            . index 3  (80)   : detector gain: average number of counts per electron
            . index 4  ('yes'): quantized electron waves result in noise
            . index 5  (0.4)  : detector quantum efficiency
            . index 6  (0.7)  : parameter of MTF
            . index 7  (0.2)  : parameter of MTF
            . index 8  (0.1)  : parameter of MTF
            . index 9  (10)   : parameter of MTF
            . index 10 (40)   : parameter of MTF
        - noise (None): if not None, overrides detector_params[4]
        - seed (-1234): seed for the run       
    """
    # see if we need overrides
    if dose is not None:
        beam_params[2]     = dose
    if defocus is not None:
        optics_params[6]   = defocus
    if noise is not None:
        detector_params[4] = noise
    # fill the dictionary
    dic = {}
    dic['simulation'] = {}
    dic['simulation']['seed']    = seed
    dic['simulation']['logfile'] = log_file
    dic['sample'] = {}
    dic['sample']['diameter']         = sample_dimensions[0] # diameter in nm
    dic['sample']['thickness_center'] = sample_dimensions[1] # thickness at center in nm
    dic['sample']['thickness_edge']   = sample_dimensions[2] # thickness at edge in nm
    dic['particle'] = {}
    dic['particle']['name']       = particle_name
    dic['particle']['voxel_size'] = voxel_size
    dic['particle']['pdb_file']   = pdb_file
    if particle_mrcout is None:
        dic['particle']['map_file_re_out'] = None
    else:
        dic['particle']['map_file_re_out'] = particle_mrcout+'_real.mrc'
        dic['particle']['map_file_im_out'] = particle_mrcout+'_imag.mrc'
    dic['particleset'] = {}
    dic['particleset']['name']     = particle_name
    dic['particleset']['crd_file'] = crd_file
    dic['beam'] = {}
    dic['beam']['voltage']      = beam_params[0] # voltage in kV
    dic['beam']['spread']       = beam_params[1] # energy spread in V
    dic['beam']['dose_per_im'] = beam_params[2] # dose per image in e/nm**2
    dic['beam']['dose_sd']      = beam_params[3] # standard deviation of dose per image
    dic['optics'] = {}
    dic['optics']['magnification']         = optics_params[0] # magnification
    dic['optics']['cs']                    = optics_params[1] # spherical aberration in mm
    dic['optics']['cc']                    = optics_params[2] # chromatic aberration in mm
    dic['optics']['aperture']              = optics_params[3] # diameter in um of aperture in back focal plane
    dic['optics']['focal_length']          = optics_params[4] # focal length in mm of primary lens
    dic['optics']['cond_ap_angle']         = optics_params[5] # aperture angle in mrad of the beam furnished by the condenser lens
    dic['optics']['defocus_nominal']       = optics_params[6] # nominal defocus value in um
    dic['optics']['defocus_syst_error']    = optics_params[7] # standard deviation of a systematic error added to the nominal defocus, measured in um. The same error is added to the defocus of every image
    dic['optics']['defocus_nonsyst_error'] = optics_params[8] # standard deviation of a nonsystematic error added to the nominal defocus and the systematic error, measured in um. A new value of the error is computed for every image
    if optics_defocout is None:
        dic['optics']['defocus_file_out']  = None
    else:
        dic['optics']['defocus_file_out']  = optics_defocout  # file to which defocus values are written
    dic['detector'] = {}
    dic['detector']['det_pix_x']        = detector_params[0] # number of pixels on detector along x axis
    dic['detector']['det_pix_y']        = detector_params[1] # number of pixels on detector along y axis
    dic['detector']['pixel_size']       = detector_params[2] # physical pixel size in um
    dic['detector']['gain']             = detector_params[3] # detector gain: average number of counts per electron
    dic['detector']['use_quantization'] = detector_params[4] # quantized electron waves result in noise
    dic['detector']['dqe']              = detector_params[5] # detector quantum efficiency
    dic['detector']['mtf_a']            = detector_params[6] # parameter of MTF
    dic['detector']['mtf_b']            = detector_params[7] # parameter of MTF
    dic['detector']['mtf_c']            = detector_params[8] # parameter of MTF
    dic['detector']['mtf_alpha']        = detector_params[9] # parameter of MTF
    dic['detector']['mtf_beta']         = detector_params[10] #parameter of MTF
    dic['detector']['image_file_out']   = mrc_file # file with resulting micrograph
    return dic               
                
def write_inp_file(numpart, pdb_file=None, crd_file=None, mrc_file=None, log_file=None, filename='input.txt', dict_params=None):
    """ write_inp_file
    """
    if mrc_file is not None and pdb_file is not None and crd_file is not None:
        if os.path.exists(mrc_file):
            print(mrc_file+' already exists.')
        else:
            #
            inp = open(filename, "w")
            inp.write('=== simulation ===\n'
                      'generate_micrographs = yes\n'
                      'rand_seed = {0[seed]}\n'
                      'log_file = {0[logfile]}\n'.format(dict_params['simulation']))
            inp.write('=== sample ===\n'
                      'diameter = {0[diameter]:d}\n'
                      'thickness_edge = {0[thickness_edge]:d}\n'
                      'thickness_center = {0[thickness_center]:d}\n'.format(dict_params['sample']))
            inp.write('=== particle {0[name]} ===\n'
                          'source = pdb\n'
                          'voxel_size = {0[voxel_size]}\n'
                          'pdb_file_in = {0[pdb_file]}\n'.format(dict_params['particle']))
            if dict_params['particle']['map_file_re_out'] is not None:
                inp.write('map_file_re_out = {0[map_file_re_out]}\n'
                          'map_file_im_out = {0[map_file_im_out]}\n'.format(dict_params['particle']))
            inp.write('=== particleset ===\n'
                      'particle_type = {0[name]}\n'
                      'particle_coords = file\n'
                      'coord_file_in = {0[crd_file]}\n'.format(dict_params['particleset']))
            inp.write('=== geometry ===\n'
                      'gen_tilt_data = yes\n'
                      'tilt_axis = 0\n'
                      'ntilts = 1\n'
                      'theta_start = 0\n'
                      'theta_incr = 0\n'
                      'geom_errors = none\n')
            inp.write('=== electronbeam ===\n'
                      'acc_voltage = {0[voltage]}\n'
                      'energy_spread = {0[spread]}\n'
                      'gen_dose = yes\n'
                      'dose_per_im = {0[dose_per_im]}\n'
                      'dose_sd = {0[dose_sd]}\n'.format(dict_params['beam']))
            inp.write('=== optics ===\n'
                      'magnification = {0[magnification]}\n'
                      'cs = {0[cs]}\n'
                      'cc = {0[cc]}\n'
                      'aperture = {0[aperture]}\n'
                      'focal_length = {0[focal_length]}\n'
                      'cond_ap_angle = {0[cond_ap_angle]}\n'
                      'gen_defocus = yes\n'
                      'defocus_nominal = {0[defocus_nominal]}\n'
                      'defocus_syst_error = {0[defocus_syst_error]}\n'
                      'defocus_syst_error = {0[defocus_nonsyst_error]}\n'.format(dict_params['optics']))
            if dict_params['optics']['defocus_file_out'] is not None:
                inp.write('defocus_file_out = {0[defocus_file_out]}\n'.format(dict_params['optics']))
            inp.write('=== detector ===\n'
                      'det_pix_x = {0[det_pix_x]}\n'
                      'det_pix_y = {0[det_pix_y]}\n'
                      'pixel_size = {0[pixel_size]}\n'
                      'gain = {0[gain]}\n'
                      'use_quantization = {0[use_quantization]}\n'
                      'dqe = {0[dqe]}\n'
                      'mtf_a = {0[mtf_a]}\n'
                      'mtf_b = {0[mtf_b]}\n'
                      'mtf_c = {0[mtf_c]}\n'
                      'mtf_alpha = {0[mtf_alpha]}\n'
                      'mtf_beta = {0[mtf_beta]}\n'
                      'image_file_out = {0[image_file_out]}\n'.format(dict_params['detector']))
            inp.close()